In [11]:
import pickle
from pathlib import Path
from collections import Counter
import pandas as pd

In [6]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
print(pv_name)
print(pv_temp)
print(pv_clean)


# !wget -N $pv_path
# # Filter for English pages, and keep just two fields: article ID (3) and monthly 
# # total number of page views (5). Then, remove lines with article id or page 
# # view values that are not a sequence of digits.
# !bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
  pickle.dump(wid2pv, f)

pageviews-202108-user.bz2
pageviews-202108-user-4dedup.txt
pageviews-202108-user.pkl


In [8]:
# read in the counter
with open(pv_clean, 'rb') as f:
  wid2pv = pickle.loads(f.read())


In [14]:
data_items = wid2pv.items()
data_list = list(data_items)[:20]
df = pd.DataFrame(data_list)
print(df)


           0      1
0    5878274     52
1    7712754  28713
2    3632887    528
3     600744   4285
4   59804426  61650
5   63743203     23
6   34443176      7
7    4838455   1450
8   11011780      9
9    4822278    779
10  34443184      4
11   2556962    594
12  59827823      1
13  59827819      2
14  39401265      1
15  10065458      6
16  55029148    123
17  61082890      1
18  16381751    209
19  55029832      4
